# Interview Coach - Демонстрация возможностей

Этот notebook показывает основные возможности системы симуляции собеседований.

In [ ]:
!pip install -r requirements.txt

## 1. Модели данных

In [ ]:
from interview_coach.models.candidate import Candidate
from interview_coach.models.feedback import FinalFeedback, Decision, Grade, HiringRecommendation, SkillAssessment, KnowledgeGap

candidate = Candidate(
    name="Коршунов Илья Константинович",
    position="Backend Developer",
    expected_grade="Junior",
    experience="1 год работы с Python и Django"
)

print("Контекст кандидата:")
print(candidate.get_context_string())

In [ ]:
decision = Decision(
    grade=Grade.JUNIOR,
    hiring_recommendation=HiringRecommendation.HIRE,
    confidence_score=75
)

skills = [
    SkillAssessment(topic="Python", evidence="Хорошо объяснил GIL", score=7),
    SkillAssessment(topic="Django", evidence="Знает ORM", score=6),
]

gaps = [
    KnowledgeGap(topic="Docker", user_answer="Не знаю", correct_answer="Контейнеризация приложений"),
]

feedback = FinalFeedback(
    decision=decision,
    confirmed_skills=skills,
    knowledge_gaps=gaps,
    summary="Хороший Junior с потенциалом роста"
)

print(feedback.to_formatted_string())

## 2. Пресеты личности кандидата

In [ ]:
from interview_coach.agents.candidate import CandidateTraits, CandidatePresets

presets = {
    "strong_junior": CandidatePresets.strong_junior(),
    "weak_junior": CandidatePresets.weak_junior(),
    "confident_wrong": CandidatePresets.confident_wrong(),
    "nervous": CandidatePresets.nervous(),
    "experienced": CandidatePresets.experienced(),
    "off_topic": CandidatePresets.off_topic(),
    "questioner": CandidatePresets.questioner(),
}

for name, traits in presets.items():
    print(f"{name}")
    print(f"Знания: {traits.knowledge_level:.1f}")
    print(f"Уверенность: {traits.confidence:.1f}")
    print(f"Честность: {traits.honesty:.1f}")
    print(f"Нервозность: {traits.nervousness:.1f}")

In [ ]:
custom_traits = CandidateTraits(
    knowledge_level=0.7,
    confidence=0.4,
    honesty=0.9,
    verbosity=0.6,
    off_topic_tendency=0.1,
    nervousness=0.5,
    curiosity=0.8
)

print("Описание личности:")
print(custom_traits.generate_personality_description())

## 3. Промпты и шаблоны

In [ ]:
from interview_coach.utils.prompts import Prompts

observer_prompt = Prompts.get_observer_prompt(
    candidate_info="Иван, Backend Developer, Junior",
    conversation_history=["Интервьюер: Что такое GIL?", "Кандидат: Global Interpreter Lock"],
    current_message="Это механизм синхронизации в CPython",
    topics_covered=["Python basics"]
)

print("Observer prompt:")
print(observer_prompt[:500] + "...")

## 4. Запуск автоматического интервью

Для запуска нужен HuggingFace API token в `.env` файле.

In [ ]:
from interview_coach.config import config

print(f"Provider: {config.llm.provider}")
print(f"Model: {config.llm.hf_model}")
print(f"API Token: {'Настроен' if config.llm.hf_api_token else 'НЕ НАСТРОЕН'}")

In [ ]:
from interview_coach.core.auto_orchestrator import run_auto_interview

feedback = run_auto_interview(
    name="Коршунов Илья Константинович",
    position="Backend Developer",
    grade="Junior",
    experience="1 год Python",
    personality="strong_junior",
    max_turns=5
)

print(feedback.to_formatted_string())

## 5. Работа с памятью диалога

In [ ]:
from interview_coach.core.memory import ConversationMemory
from interview_coach.models.turn import Turn, ObserverAnalysis, ResponseQuality, ConfidenceLevel

memory = ConversationMemory()

for i in range(3):
    analysis = ObserverAnalysis(
        quality_score=6 + i,
        quality_level=ResponseQuality.GOOD,
        confidence=ConfidenceLevel.MEDIUM,
        topics_mentioned=[f"Topic_{i}"],
        recommendation="Продолжать"
    )
    turn = Turn(
        turn_id=i+1,
        agent_visible_message=f"Вопрос {i+1}",
        user_message=f"Ответ {i+1}",
        observer_analysis=analysis
    )
    memory.add_turn(turn)

print(f"Количество ходов: {memory.get_turn_count()}")
print(f"Средний скор: {memory.get_average_score():.1f}")
print(f"Темы: {memory.topics_covered}")
print(f"Увеличить сложность: {memory.should_increase_difficulty()}")

## 6. Анализ ответов (Turn models)

In [ ]:
from interview_coach.models.turn import ObserverAnalysis, InterviewerInstruction, ResponseQuality, ConfidenceLevel

analysis = ObserverAnalysis(
    quality_score=8,
    quality_level=ResponseQuality.GOOD,
    confidence=ConfidenceLevel.HIGH,
    hallucination_detected=True,
    hallucination_details="Python 3 вышел в 2010, не 2008",
    correct_points=["Синтаксис", "ООП"],
    incorrect_points=["Дата релиза"],
    recommendation="Уточнить даты"
)

print("Внутренние мысли Observer:")
print(analysis.to_internal_thoughts())

In [ ]:
instruction = InterviewerInstruction(
    action="correct",
    difficulty_level=3,
    correction_needed=True,
    correction_text="Python 3 был выпущен в декабре 2008 года"
)

print("Внутренние мысли Interviewer:")
print(instruction.to_internal_thoughts())